# Среднее число жителей в заселённых квартирах

PPF — People Per Flat. Характеризует только те квартиры, в которых кто-то постоянно живёт. Соответственно, если в квартире никто постоянно не живёт, то для неё этот показатель неприменим. Благодаря этой особенности можно использовать PPF для оценки вакантности. Как именно — см. тетрадку №4.

PPF считаем по данным [переписи 2021 года в Москве](https://77.rosstat.gov.ru/folder/210976). В томе 11 есть таблица 6: «Жилые помещения по их типам, числу комнат и числу проживающих в них частных домохозяйств по муниципальным образованиям». Из неё можно понять, сколько человек в среднем проживает в одной квартире с учётом числа комнат по муниципалитетам. Расчёт получается не самый тривиальный, так как данные идут в разбивке по (1) числу домохозяйств в одной квартире и (2) числу проживающих в домохозяйстве, но если всё аккуратно перемножить и сложить, а потом поделить (формула есть в коде), то получим среднее число постоянно проживающих в одной заселённой квартире в разбивке по муниципалитетам и числу комнат в квартире (1–4, `-1` значит среднее независимо от числа комнат). Разбивка по числу комнат важна, чтобы получить более точный результат — в тетрадке №3 оцениваем число квартир с разным числом комнат по домам.

Так как данные переписи собираются bottom-up, то есть от конкретных домохозяйств, то пустующие квартиры в указанной выше таблице не учитываются. Таким образом, мы, грубо говоря, получаем косвенный результат поквартирного обхода и на его основе пытаемся потом определить исходные данные по вакантности. Из методики переписи мы также получаем некоторое определение вакантности: это жилое помещение, в котором постоянно никто не живёт. Подробнее см. [методологические пояснения к переписи](https://rosstat.gov.ru/storage/mediabank/%D0%9E%D0%B1%D1%89%D0%B0%D1%8F%20%D1%87%D0%B0%D1%81%D1%82%D1%8C_Met_VPN-2020.docx), с. 3 с «Население переписано по месту своего постоянного (обычного) жительства».

In [4]:
from fuzzywuzzy import process
import pandas as pd

`liv_cond` — результаты переписи, `repair_data` — набор открытых данных ФРТ «[Многоквартирные дома в региональной программе капитального ремонта по городу Москве (отчет КР 1.1)](https://xn--80adsazqn.xn--p1aee.xn--p1ai/opendata/export/184)». Используем именно их, а не те же, что в геокодировании, так как здесь есть разбивка по муниципальным образованиям. Набор нужен, чтобы привести наименования муниципалитетов к тем, что используются в открытых данных ФРТ, с которыми мы дальше работаем.

In [8]:
liv_cond = pd.read_excel("data/census_results_11_6.xlsx", sheet_name=None, na_values=["-"])
repair_data = pd.read_csv("data/export-kr1_1-77-20240701.zip", sep=";")

In [10]:
mo = repair_data["mun_obr"].unique()

In [23]:
MO_MAPPING = {
    "Савеловское": "Савёловский район",
    "Нагорное": "Нагорный район",
    "Северное": "район Северный",
    "Восточное": "район Восточный",
    "Хорошево-Мневники": "район Хорошёво-Мнёвники",
    "Бабушкинское": "Бабушкинский район" ,
    "Поселение Троицк": "городской округ Троицк",
    "Поселение Щербинка": "городской округ Щербинка",
    "Хорошевское": "Хорошёвский район",
    "Пресненское": "Пресненский район",
    "Рязанское": "Рязанский район",
    "Измайлово": "район Измайлово"
} # manual fixes of wrong fuzzy search results

parts = []
for sheet_name, data in liv_cond.items():
    mun_obr = MO_MAPPING.get(sheet_name)
    if mun_obr is None:
        mun_obr = process.extractOne(sheet_name, mo, score_cutoff=70)
        if mun_obr is None:
            print(f"Cannot find matching municipality for {sheet_name}")
            continue
        else:
            mun_obr = mun_obr[0]
    
    part = data.iloc[[17, 19, 20, 21, 22]].copy()
    part.columns = [f"c{i}" for i in range(part.shape[1])]
    part["ppf"] = (
        (
            part["c3"] + part["c4"] * 2 + part["c5"] * 3 + part["c6"] * 4 + part["c8"]
             + part["c10"] + part["c12"] + part["c14"]
        ) / part["c1"]
    )        
    part["mun_obr"] = mun_obr
    part["sheet_name"] = sheet_name
    part["n_rooms"] = [-1] + list(range(1, 5))    
    parts.append(part[["mun_obr", "ppf", "n_rooms", "sheet_name"]])

ppf_by_mo = pd.concat(parts)

Cannot find matching municipality for г. Москва
Cannot find matching municipality for Муниципальные образования ВАО
Cannot find matching municipality for Муниципальные образования ЗАО
Cannot find matching municipality for Муниципальные образования ЗелАО
Cannot find matching municipality for Муниципальные образования САО
Cannot find matching municipality for Муниципальные образования СВАО
Cannot find matching municipality for Муниципальные образования СЗАО
Cannot find matching municipality for Муниципальные образования ЦАО
Cannot find matching municipality for Муниципальные образования ЮВАО
Cannot find matching municipality for Муниципальные образования ЮЗАО
Cannot find matching municipality for Муниципальные образования ЮАО
Cannot find matching municipality for Муниципальные образования НАО
Cannot find matching municipality for Муниципальные образования ТАО


Листы, соответствующие более крупным территориальным единицам, ожидаемо пропущены, а остальные обработаны. Потом ещё вручную проверил правильность сопоставления муниципалитетов, так как автоматический поиск через `fuzzywuzzy` не всегда даёт правильный результат.

In [24]:
ppf_by_mo.head(10)

,mun_obr,ppf,n_rooms,sheet_name
17,район Богородское,2.621264,-1,Богородское
19,район Богородское,2.004463,1,Богородское
20,район Богородское,2.669125,2,Богородское
21,район Богородское,3.343224,3,Богородское
22,район Богородское,3.848692,4,Богородское
17,район Вешняки,2.703723,-1,Вешняки
19,район Вешняки,2.36266,1,Вешняки
20,район Вешняки,2.697963,2,Вешняки
21,район Вешняки,3.351128,3,Вешняки
22,район Вешняки,4.033898,4,Вешняки


In [25]:
ppf_by_mo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 730 entries, 17 to 22
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   mun_obr     730 non-null    object
 1   ppf         703 non-null    object
 2   n_rooms     730 non-null    int64 
 3   sheet_name  730 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.5+ KB


Пропуски возникли из-за того, что по некоторым муниципальных образованиях не было каких-то данных переписи. Ничего с ними не делаем, так как их мало. Сохраняем результат для дальнейшего использования.

In [26]:
ppf_by_mo.to_csv("data/ppf_by_mo.csv", index=False)